In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('../data/csv/cleaned_final_csv_scrap_completo.csv', encoding='utf-8')

#### df info

In [ ]:
df.info()

#### ML

In [ ]:
df['Model'].value_counts().head(20)

In [ ]:
cols = ['ID_Auto', 'Brand', 'Tipo', 'Sucursal', 'Year', 'Km', 'Caja', 'Precio']
df_ml = df[cols].copy()
df_ml = df_ml.drop_duplicates(['ID_Auto'])

print(df_ml.nunique())
df_ml = df_ml.drop(columns=['ID_Auto'])

# Convertir ['Caja', 'Sucursal'] a columnas numericas
df_ml = pd.get_dummies(df_ml, columns=['Caja', 'Sucursal', 'Tipo'], drop_first=True)

In [ ]:
X = df_ml.drop(columns=['Precio'])
y = df_ml['Precio']

In [ ]:
### Dividir datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Entranamiento: {X_train.shape[0]} autos")
print(f"Examen (Test): {X_test.shape[0]} autos")

In [ ]:
### Target Encoding

# Unir los precios temporalmente para calcular los promedios
train_temp = X_train.copy()
train_temp['Precio_Real'] = y_train
# Calcular precios promedio para cada Marca y Modelo
brand_map = train_temp.groupby('Brand')['Precio_Real'].mean() # {'marca': #####}
#model_map = train_temp.groupby('Model')['Precio_Real'].mean() # {'modelo': #####}
global_mean = y_train.mean() 
# Remplazo de marcas y modelos por precio promedio en train
X_train['Brand_Encoded'] = X_train['Brand'].map(brand_map) #VLOOKUP para precio promedio de cada marca
#X_train['Model_Encoded'] = X_train['Model'].map(model_map) #VLOOKUP para precio promedio de cada modelo

# Remplazo de marcas y modelos por precio promedio en test
X_test['Brand_Encoded'] = X_test['Brand'].map(brand_map)
#X_test['Model_Encoded'] = X_test['Model'].map(model_map)

# Limpieza de nulos por media de cada marca por si hay un modelo que no estaba en train
X_test['Brand_Encoded'] = X_test['Brand_Encoded'].fillna(global_mean)
# X_test['Model_Encoded'] = X_test['Model_Encoded'].fillna(global_mean)

# Eliminar columnas originales
X_train = X_train.drop(columns=['Brand'])
X_test = X_test.drop(columns=['Brand'])

In [ ]:
print(X_train.columns)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

In [ ]:
### Entrenamiento
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
### Prediccion
y_pred = rf_model.predict(X_test)

In [ ]:
### Evaluacion
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
# Calculo del Error Porcentual Promedio (MAPE)
errores = abs(y_test - y_pred)
mape = 100 * np.mean(errores / y_test)

In [ ]:
print("\n--- RESULTADOS DEL MODELO ---")
print(f"MAE (Error Promedio en Pesos):  ${mae:,.2f} MXN")
print(f"MAPE (Error Promedio %):        {mape:.2f}%")
print(f"R2 (Precisión General):         {r2:.2f}")

In [ ]:
print("\n--- EJEMPLO REAL ---")
# Comparemos el primer auto del Test Set
precio_real = y_test.iloc[0]
precio_predicho = y_pred[0]
diferencia = precio_predicho - precio_real

print(f"Auto de prueba #1:")
print(f"Precio Real Kavak:   ${precio_real:,.2f}")
print(f"El Modelo predice:   ${precio_predicho:,.2f}")
print(f"Diferencia:          ${diferencia:,.2f}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Extraemos la importancia de cada variable
importances = rf_model.feature_importances_
feature_names = X_train.columns

# Creamos una tabla
df_imp = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
df_imp = df_imp.sort_values('Importance', ascending=False)

# Mostramos el TOP 5
print(df_imp.head(5))

# Graficamos
plt.figure(figsize=(10, 6))
plt.barh(df_imp['Feature'].head(10), df_imp['Importance'].head(10), color='salmon')
plt.xlabel('Importancia (0 a 1)')
plt.title('¿Quién le sopló la respuesta al modelo?')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
import pandas as pd

# 1. Configuración del Experimento
# Usaremos el promedio de tu marca más común (Brand_Encoded) para no sesgar
brand_mean_val = X_train['Brand_Encoded'].mean()

# Creamos un auto base: 2020, 40,000 km, Automático, en CDMX
# (Asegúrate de que los nombres de columnas coincidan con tus dummies del print(X_train.columns))
auto_base = {col: 0 for col in X_train.columns} # Todo en ceros al inicio

auto_base['Year'] = 2020
auto_base['Km'] = 40000
auto_base['Brand_Encoded'] = brand_mean_val
if 'Caja_Manual' in auto_base: auto_base['Caja_Manual'] = 0 # Es Automático
if 'Sucursal_Ciudad de México' in auto_base: auto_base['Sucursal_Ciudad de México'] = 1

# --- AUTO 1: EL SEDÁN ---
df_sedan = pd.DataFrame([auto_base.copy()])
if 'Tipo_Sedan' in df_sedan.columns:
    df_sedan['Tipo_Sedan'] = 1

# --- AUTO 2: LA SUV (CAMIONETA) ---
df_suv = pd.DataFrame([auto_base.copy()])
if 'Tipo_Suv' in df_suv.columns:
    df_suv['Tipo_Suv'] = 1

# 3. PREDICCIÓN
print("\n---PELEA DE PRECIOS: SEDÁN VS SUV (Misma Marca/Año/Km) ---")
try:
    precio_sedan = rf_model.predict(df_sedan)[0]
    precio_suv = rf_model.predict(df_suv)[0]
    diferencia = precio_suv - precio_sedan
    
    print(f"Precio Sedán: ${precio_sedan:,.2f}")
    print(f"Precio SUV:   ${precio_suv:,.2f}")
    print(f"Diferencia:   ${diferencia:,.2f}")
    
    if diferencia > 15000:
        print("VEREDICTO: El modelo APRENDIÓ que el tipo de carrocería importa.")
    else:
        print("VEREDICTO: El modelo NO distingue tipos. El precio depende solo de la Marca.")

except Exception as e:
    print(f"Error en columnas: {e}")